In [39]:
import numpy as np
import pandas as pd
import scipy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import QuantileTransformer, KBinsDiscretizer
from sklearn.model_selection import StratifiedKFold, train_test_split, KFold
from torch.optim.lr_scheduler import CosineAnnealingLR, OneCycleLR
from tqdm import tqdm
import torchmetrics
from torchmetrics import AUROC
from tab_transformer_pytorch import TabTransformer
import matplotlib.pyplot as plt
import gc, sys, random, warnings
gc.enable()
warnings.filterwarnings("ignore")

In [2]:
torchmetrics.__version__

'1.3.1'

In [3]:
pd.set_option('display.max_columns',1000)

In [4]:
torch.__version__

'1.11.0'

In [5]:
def seed_all(seed_value):
    random.seed(seed_value) # Python
    np.random.seed(seed_value) # cpu vars
    torch.manual_seed(seed_value) # cpu  vars
    
    if torch.cuda.is_available(): 
        torch.cuda.manual_seed(seed_value)
        torch.cuda.manual_seed_all(seed_value) # gpu vars
        torch.backends.cudnn.deterministic = True  #needed
seed_all(123)

In [6]:
# Load data
path = './Backup_Regression/'
name = 'Predictions_Option_1.xlsx'

data = pd.read_excel(path+name)
print(data.shape)

(738, 39)


In [7]:
# bmi_original_columns = [col for col in data.columns if 'Body mass index' in col and 'after surgery' in col and 'predicted' not in col]
post_surgery_columns = [col for col in data.columns if 'after surgery' in col and 'predicted' not in col and 'Diabetes Mellitus' not in col]
print(post_surgery_columns)

# Drop the bmi post surgery (original) columns, since a lot of them have mising values
cols_to_drop = post_surgery_columns + ['record_id','redcap_data_access_group']
print(f'Dropping {len(cols_to_drop)} columns...')
data = data.drop(columns=cols_to_drop)
#print('Adding a sample weight column..')
#data['sample_weight'] = 1
print(data.shape)

data.columns = [col.strip() for col in data.columns]

data4 = data.copy()
data5 = data4[(data4['Diabetes Mellitus 3 months after surgery (1 = Yes\n2 = No)'].isnull() == False)&(data4['Diabetes Mellitus 6 months after surgery (1 = Yes\n2 = No)'].isnull() == False)&(data4['Diabetes Mellitus 12 months after surgery (1 = Yes\n2 = No)'].isnull() == False)&(data4['Diabetes Mellitus 18 months after surgery (1 = Yes\n2 = No)'].isnull() == False)&(data4['Diabetes Mellitus 2 years after surgery (1 = Yes\n2 = No)'].isnull() == False)&(data4['Diabetes Mellitus 3 years after surgery (1 = Yes\n2 = No)'].isnull() == False)&(data4['Diabetes Mellitus 4 years after surgery (1 = Yes\n2 = No)'].isnull() == False)&(data4['Diabetes Mellitus 5 years after surgery (1 = Yes\n2 = No)'].isnull() == False)].copy()
data6 = data4.loc[~data4.index.isin(data5.index.tolist())]
print(data4.shape, data5.shape, data6.shape)

# Split the data in train and validation and then, for train data, perform the KFold Cross Validation
target_columns = ['Diabetes Mellitus 3 months after surgery (1 = Yes\n2 = No)',
                 'Diabetes Mellitus 6 months after surgery (1 = Yes\n2 = No)',
                 'Diabetes Mellitus 12 months after surgery (1 = Yes\n2 = No)',
                 'Diabetes Mellitus 18 months after surgery (1 = Yes\n2 = No)',
                 'Diabetes Mellitus 2 years after surgery (1 = Yes\n2 = No)',
                 'Diabetes Mellitus 3 years after surgery (1 = Yes\n2 = No)',
                 'Diabetes Mellitus 4 years after surgery (1 = Yes\n2 = No)',
                 'Diabetes Mellitus 5 years after surgery (1 = Yes\n2 = No)']

Y = data5[target_columns].copy()
# Replace 2's in Y with 0's
Y = Y.replace(2,0)
X = data5.drop(columns = target_columns).copy()
X1 , X_val , Y1 , Y_val = train_test_split(X ,
                                           Y,
                                           test_size = 0.2,
                                           random_state = 1234)

print(X1.shape, Y1.shape, X_val.shape, Y_val.shape)

# train_data = pd.concat([X1,Y1],axis=1)
# test_data = pd.concat([X_val,Y_val],axis=1)
# print(train_data.shape, test_data.shape)


target_df = Y1
train_df = X1
#train_df['target'] = target_df.iloc[:,:1]
test_df = X_val
print(train_df.shape, target_df.shape, test_df.shape)

['Complication after surgery (1 = Anastomotic leakage\n2 = Sleeve leak\n3 = Intussusception after Roux-en-Y gastric bypass\n4 = Mesenteric internal hernia after Roux-en-Y gastric bypass\n5 = Internal hernia through Peterson`s defect after Roux-en-Y gastric bypass\n6 = Hiatal hernia\n7 = Gastro Esophageal Reflux Disease (GERD)\n8 = Amastomotic Ulcer\n9 = Anastomotic stricture\n10 = Hemorrhage\n11 = No Complication)', 'Body mass index 3 months after surgery', 'Body mass index 6 months after surgery', 'Body mass index 12 months after surgery', 'Body mass index 18 months after surgery', 'Body mass index 2 years after surgery', 'Body mass index 3 years after surgery', 'Body mass index 4 years after surgery', 'Body mass index 5 years after surgery']
Dropping 11 columns...
(738, 28)
(738, 28) (738, 28) (0, 28)
(590, 20) (590, 8) (148, 20) (148, 8)
(590, 20) (590, 8) (148, 20)


In [8]:
cat_cols = [
    'sex (1=Female\n2=Male)',
    'American Society of Anesthesiologists (ASA) Score (1 = ASA 1: healthy person\n2 = ASA 2: mild systemic disease\n3 = ASA 3: severe systemic disease\n4 =ASA 4: severe systemic disease that is a constant threat to life\n5 = ASA 5: a moribund person who is not expected to survive without the operation)',
    'Diabetes mellitus type II preoperative (1 = yes\n2 = No)',
    'Antidiabetic drugs preoperativ (1 = Orale Antidiabetic drugs\n2 = Insulin\n3 = No therapy)',
    'Obstructive sleep apnea syndrome (OSAS) preoperative  (1 = yes\n2 = No)',
    'surgery (1 = LSG (Laparoscopic Sleeve Gastrectomy)\n2 = RYGB (Roux en-Y Gastric Bypass)\n3 = SADI (single anastomosis duodeno-ileal bypass)\n4 = BPD-DS (Biliopancreatic diversion with duodenal switch)\n5 = OAGB (One-anastomosis Gastric Bypass)',
    'Conversion from gastric sleeve to gastric bypass (1 = Yes\n2 = No\n3 = Not available)',
    "Closure of Petersen's space (1 = Yes\n2 = No\n3 = Not available)",
    'Closure of the jejunojejunostomy defect (1 = Yes\n2 = No\n3 = Not available)'
]

cont_cols = list(set(train_df.columns)-set(cat_cols))

train_df = train_df[cont_cols+cat_cols]
test_df = test_df[cont_cols+cat_cols]

##### Converting numerical features into Normal Distributions

In [9]:
train_scaled, test_scaled = train_df.copy(), test_df.copy()
for col in cont_cols:
    transformer = QuantileTransformer(n_quantiles=100,random_state=0, output_distribution="normal")
    vec_len = len(train_scaled[col].values)
    vec_len_test = len(test_scaled[col].values)
    raw_vec = train_scaled[col].values.reshape(vec_len, 1)
    transformer.fit(raw_vec)

    train_scaled[col] = transformer.transform(raw_vec).reshape(1, vec_len)[0]
    test_scaled[col] = transformer.transform(test_scaled[col].values.reshape(vec_len_test, 1)).reshape(1, vec_len_test)[0]

In [10]:
del(train_df); del (test_df); gc.collect()

104

##### Discretizing Continuous Features

In [11]:
disc = KBinsDiscretizer(n_bins=50, encode='ordinal',strategy='uniform')
train_scaled[cont_cols] = disc.fit_transform(train_scaled[cont_cols])
test_scaled[cont_cols] = disc.transform(test_scaled[cont_cols])

In [13]:
y_train = target_df.values
train_scaled = train_scaled.values
test_scaled = test_scaled.values

In [ ]:
class_weights = torch.tensor([0.16, 0.16, 0.25, 0.25, 0.083, 0.083])

In [118]:
y_train.shape[0]

590

In [123]:
list(1/y_train.sum(axis=0))

[0.02040816326530612,
 0.019230769230769232,
 0.012345679012345678,
 0.023255813953488372,
 0.01282051282051282,
 0.022727272727272728,
 0.025,
 0.024390243902439025]

In [15]:
y_train.shape, train_scaled.shape, test_scaled.shape

((590, 8), (590, 20), (148, 20))

##### Utilities

In [16]:
class AverageMeter:
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

In [18]:
class EarlyStopping:
    def __init__(self, patience=7, mode="max", delta=0.001, verbose = None):
        self.patience = patience
        self.counter = 0
        self.mode = mode
        self.best_score = None
        self.early_stop = False
        self.delta = delta
        self.verbose = verbose
        if self.mode == "min":
            self.val_score = np.Inf
        else:
            self.val_score = -np.Inf

    def __call__(self, epoch_score, model, model_path):

        if self.mode == "min":
            score = -1.0 * epoch_score
        else:
            score = np.copy(epoch_score)

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
        elif score < self.best_score: #  + self.delta
            self.counter += 1
            if self.verbose:
                print('EarlyStopping counter: {} out of {}'.format(self.counter, self.patience))
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(epoch_score, model, model_path)
            self.counter = 0

    def save_checkpoint(self, epoch_score, model, model_path):
        if epoch_score not in [-np.inf, np.inf, -np.nan, np.nan]:
            if self.verbose:
                print('Validation score improved ({:.4f} --> {:.4f}). Saving model!'.format(self.val_score, epoch_score))
                
            torch.save(model.state_dict(), model_path)
        self.val_score = epoch_score

In [46]:
# Updated to pass a contant cont value in this version
class TabDataset(Dataset):
    def __init__(self, cat, target = None):
        super().__init__()
        self.cat = cat
        self.target = target
        
    def __len__(self):
        return len(self.cat)
    
    def __getitem__(self, idx):
        cat = self.cat[idx]
        
        _dict = {'cont': torch.ones(1),
                 'cat': torch.LongTensor(cat)}
        
        if self.target is not None:
            target = self.target[idx]
            _dict.update({'target': torch.tensor(target, dtype = torch.float)})
        
        return _dict

In [34]:
class Trainer:
    def __init__(self, model, device, loss_fn, opt, scheduler = None):
        self.model = model
        self.device = device
        self.loss_fn = loss_fn
        self.opt = opt
        self.scheduler = scheduler
        
    def fit_one_epoch(self, dl):
        self.model.train()
        losses = AverageMeter()
        prog_bar = tqdm(enumerate(dl), total = len(dl), file=sys.stdout, leave = False)
        
        for bi, d in prog_bar:
            cont = d["cont"].to(self.device)
            cat = d['cat'].to(self.device)
            target = d['target'].to(self.device)
            
            out = self.model(cat, cont)
            loss = self.loss_fn(out.squeeze(-1), target)
            prog_bar.set_description('loss: {:.2f}'.format(loss.item()))
            losses.update(loss.item(), cont.size(0))
            loss.backward()
            self.opt.step()
            
            if self.scheduler: 
                self.scheduler.step()
                    
            self.opt.zero_grad()
            
    def eval_one_epoch(self, dl, **kwargs):
        self.model.eval()
        losses = AverageMeter()
        metric = AUROC(task='binary')
        prog_bar = tqdm(enumerate(dl), total = len(dl), file=sys.stdout, leave = False)
        
        for bi, d in prog_bar:  
            cont = d["cont"].to(self.device)
            cat = d['cat'].to(self.device)
            target = d['target'].to(self.device)
            
            with torch.no_grad():
                out = self.model(cat, cont)
                #print('Out :', out)
                loss = self.loss_fn(out.squeeze(-1), target)
                if metric:
                    auroc = metric(out.squeeze(-1), target.int())
                
                losses.update(loss.item(), cont.size(0))
        auroc = metric.compute()
        print(f"F{kwargs['fold']} E{str(kwargs['epoch']):2s}"\
              f"  Valid Loss: {losses.avg:.4f}  AUROC Score: {auroc:.4f}")
        return auroc.cpu() if metric else losses.avg

##### Training

In [ ]:
kfold.split()

In [27]:
class cfg:
    bs = 64
    n_splits = 5
    seed = 2021
    epochs = 3
    lr = 2e-5
    checkpoint = lambda fold: f'full_cat_{fold}.pt'
    
kfold = KFold(n_splits = cfg.n_splits, 
                        random_state = cfg.seed, 
                        shuffle = True)
splits = [*kfold.split(X = train_scaled, y = y_train[:,:1])]

In [28]:
splits[0]

(array([  2,   4,   5,   7,   9,  10,  11,  12,  15,  16,  17,  18,  19,
         20,  22,  23,  25,  26,  27,  29,  30,  31,  33,  35,  37,  38,
         39,  40,  41,  42,  43,  44,  45,  47,  48,  49,  50,  51,  53,
         54,  55,  56,  57,  58,  59,  61,  62,  63,  64,  65,  66,  67,
         68,  69,  70,  71,  72,  74,  75,  77,  78,  79,  80,  81,  82,
         83,  84,  85,  86,  87,  89,  91,  92,  93,  94,  95,  96,  97,
         99, 100, 101, 102, 104, 105, 107, 108, 109, 110, 112, 113, 114,
        116, 117, 119, 120, 121, 122, 123, 126, 127, 128, 129, 130, 131,
        132, 133, 134, 135, 136, 137, 138, 139, 140, 142, 143, 144, 145,
        146, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 159, 160,
        161, 162, 163, 164, 166, 167, 168, 169, 170, 171, 172, 173, 174,
        175, 176, 178, 180, 181, 183, 184, 185, 186, 188, 189, 190, 191,
        192, 193, 194, 195, 196, 197, 198, 199, 201, 202, 203, 204, 205,
        206, 208, 211, 212, 213, 215, 216, 217, 218

In [29]:
len(cat_cols), len(cont_cols)

(9, 11)

In [45]:
y_train[0,0].item()

0

In [30]:
transformer_cfg = {
    'categories' : [50]*11 + [2, 5, 2, 3, 2, 5, 3, 3, 3],           # iterable with the number of unique values for categoric feature
    #'categories' : [2, 5, 2, 3, 2, 5, 3, 3, 3],           # iterable with the number of unique values for categoric feature
    'num_continuous' : 1,                       # continuous dimensions in data
    'dim' : 32,                                 # hidden dim, paper set at 32
    'dim_out' : 8,                              # binary prediction
    'depth' : 3,                                # depth, paper recommended 6
    'heads' : 6,                                # heads, paper recommends 8
    'attn_dropout' : 0.1,                       # post-attention dropout
    'ff_dropout' : 0.1,                         # feed forward dropout
    'mlp_hidden_mults' : (4, 2),                # relative multiples of each hidden dimension of the last mlp to logits
    'mlp_act' : nn.GELU(),                      # activation for final mlp, defaults to relu
    'continuous_mean_std' : torch.randn(1, 2)   # normalize the continuous values before layer norm (optional)
}

In [31]:
# device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

In [32]:
def create_dataloaders(fold):
    train_idx, valid_idx = splits[fold]
    
    _xtr, _ytr = train_scaled[train_idx], y_train[train_idx]
    _xval, _yval = train_scaled[valid_idx], y_train[valid_idx]
    
    train_ds = TabDataset(cat = _xtr, target = _ytr)
    valid_ds = TabDataset(cat = _xval, target = _yval)
                          
    train_dl = DataLoader(train_ds, batch_size = cfg.bs, shuffle = True)
    valid_dl = DataLoader(valid_ds, batch_size = cfg.bs, shuffle = False)
    
    return train_dl, valid_dl

In [35]:
def train_fold(fold, epochs = 20):
    train_dl, valid_dl = create_dataloaders(fold)
    es = EarlyStopping(patience = 7, mode="max", verbose = False)
    
    model = TabTransformer(**transformer_cfg).to(device)
    
    opt = torch.optim.AdamW(model.parameters(), lr = cfg.lr)

    trainer = Trainer(model, 
                      device, 
                      loss_fn=nn.BCEWithLogitsLoss(),
                      opt = opt,
                      scheduler = None,
                     )
    
    for epoch in range(epochs):
        trainer.fit_one_epoch(train_dl)
        valid_loss = trainer.eval_one_epoch(valid_dl, fold = fold, epoch = epoch)
        
        es(valid_loss, trainer.model, model_path = cfg.checkpoint(fold))       
        if es.early_stop:
            break

In [47]:
for fold in range(cfg.n_splits):
    train_fold(fold, cfg.epochs)
    torch.cuda.empty_cache()
    gc.collect()

F0 E0   Valid Loss: 0.6051  AUROC Score: 0.7529                                                                        
F0 E1   Valid Loss: 0.5275  AUROC Score: 0.8288                                                                        
F0 E2   Valid Loss: 0.4547  AUROC Score: 0.8623                                                                        
F1 E0   Valid Loss: 0.5951  AUROC Score: 0.7106                                                                        
F1 E1   Valid Loss: 0.5104  AUROC Score: 0.8264                                                                        
F1 E2   Valid Loss: 0.4301  AUROC Score: 0.8837                                                                        
F2 E0   Valid Loss: 0.6017  AUROC Score: 0.7191                                                                        
F2 E1   Valid Loss: 0.5144  AUROC Score: 0.8452                                                                        
F2 E2   Valid Loss: 0.4307  AUROC Score:

##### Prediction

In [49]:
y_pred = torch.zeros(len(test_scaled), 8).to(device)
test_ds = TabDataset(cat = test_scaled)
test_dl = DataLoader(test_ds, batch_size = cfg.bs, shuffle = False)

with torch.no_grad():
    for fold in range(cfg.n_splits):
        preds = []
        model = TabTransformer(**transformer_cfg).to(device)
        state_dict = cfg.checkpoint(fold)
        model.load_state_dict(torch.load(state_dict))
        model.eval()
        
        for d in test_dl:
            cont = d["cont"].to(device)
            cat = d['cat'].to(device)
            out = model(cat, cont)
            preds.append(out)
            
        preds = torch.vstack(preds)
        y_pred += preds / cfg.n_splits

In [51]:
y_pred.shape

torch.Size([148, 8])

In [55]:
torch.tensor(Y_val.values).shape

torch.Size([148, 8])

In [60]:
y_pred

tensor([[-0.7919, -0.7902, -0.7587,  ..., -0.8591, -0.8288, -0.8651],
        [-0.7902, -0.8099, -0.7678,  ..., -0.9030, -0.8288, -0.8148],
        [-0.6264, -0.6321, -0.5157,  ..., -0.7070, -0.6685, -0.7639],
        ...,
        [-0.7343, -0.7299, -0.6506,  ..., -0.8169, -0.8064, -0.7835],
        [-0.8713, -0.8673, -0.8537,  ..., -0.9560, -0.9312, -0.9130],
        [-0.6894, -0.7314, -0.7007,  ..., -0.8131, -0.7698, -0.7483]])

In [62]:
Y_val.values

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 1, ..., 1, 1, 1],
       ...,
       [1, 0, 1, ..., 1, 1, 1],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

In [63]:
loss_fn=nn.BCEWithLogitsLoss()
loss_fn(y_pred, torch.tensor(Y_val.values, dtype=torch.float32))
# nn.BCEWithLogitsLoss()

tensor(0.4377)

In [70]:
sigmoid = nn.Sigmoid()
y_hat = sigmoid(y_pred)

In [75]:
pd.DataFrame(y_hat).describe()

,0,1,2,3,4,5,6,7
count,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000,148.000000
mean,0.310236,0.308740,0.320185,0.300516,0.323239,0.291340,0.299247,0.299631
std,0.018310,0.018796,0.022429,0.015660,0.020792,0.017816,0.016359,0.016108
min,0.276965,0.282198,0.283236,0.278874,0.287528,0.263196,0.268981,0.272665
25%,0.297601,0.294224,0.303911,0.289724,0.309635,0.278355,0.287337,0.288586
50%,0.305068,0.303543,0.313356,0.296954,0.316957,0.287084,0.295840,0.296519
75%,0.318989,0.319847,0.329614,0.307485,0.330656,0.299967,0.308041,0.309522
max,0.365376,0.367900,0.395372,0.356913,0.387490,0.350941,0.349027,0.347511


In [69]:
y_pred[0]

tensor([-0.7919, -0.7902, -0.7587, -0.8137, -0.7485, -0.8591, -0.8288, -0.8651])

In [66]:
y_pred[0,:]

tensor([-0.7919, -0.7902, -0.7587, -0.8137, -0.7485, -0.8591, -0.8288, -0.8651])

In [107]:
target = torch.tensor([[0,1,0,1,0,0],[0,1,0,1,0,1]], dtype=torch.float32)
output = torch.tensor([[0.2,0.9,0.2,0.8,0.3,0.2],[0.1,0.8,0.2,0.9,0.3,0.7]])
weights = torch.tensor([0.16, 0.16, 0.25, 0.25, 0.083, 0.083])

In [108]:
criterion = nn.BCEWithLogitsLoss(reduction='none')
loss = criterion(output, target)

In [109]:
loss

tensor([[0.7981, 0.3412, 0.7981, 0.3711, 0.8544, 0.7981],
        [0.7444, 0.3711, 0.7981, 0.3412, 0.8544, 0.4032]])

In [110]:
loss

tensor([[0.7981, 0.3412, 0.7981, 0.3711, 0.8544, 0.7981],
        [0.7444, 0.3711, 0.7981, 0.3412, 0.8544, 0.4032]])

In [111]:
loss = (loss * weights).mean()

In [112]:
loss

tensor(0.0983)

In [101]:
loss

tensor(0.0983)